In [48]:
from DataLoader import importar_ventas,leer_csv,agregar_fourier,preparar_test
from DataLoader import generar_rolling_features,crear_archivo_kaggle
import numpy as np
from sklearn.model_selection import GridSearchCV,PredefinedSplit, RandomizedSearchCV
from xgboost import XGBRegressor
import pandas as pd

In [ ]:
def predecir(separador):

In [60]:
ventas=importar_ventas()
ventas
agregar_fourier(ventas,2)
ventas=generar_rolling_features(ventas,"demand",[1,4],["subgroup_cod","store_cod"])

In [5]:
print(len(ventas["group"].unique()))
for group in ventas["group"].unique():
    print(len(ventas[ventas["group"]==group])) #tambien podria ser

23
188989
208328
212403
323148
359052
258450
184253
204801
184564
174080
102800
173558
210201
209727
366582
189105
194389
223177
344324
271314
279056
161916
182576


In [61]:
#model=XGBRegressor()
separador="group"
bins=ventas[separador].unique()
TARGET=["demand"]
FEATURES=[#'mean_price', #'std_price', 
          'max_price', 'min_price',
        'cluster', #'store_cod', 
        # 'subgroup_cod', #'day',
       #'month', 
        'year', 'anual_sin_1',
    'anual_cos_1', 'mensual_sin_1',
       #'mensual_cos_1', 
        #'anual_sin_2', 'anual_cos_2', 
    'mensual_sin_2', #'demand_rolling_mean_7d',
       'demand_rolling_sum_7d', 
    #'demand_rolling_mean_28d',
       #'demand_rolling_sum_28d'
       #'mensual_cos_2',
        # 'mean_price_rolling_mean_7d',
       #'mean_price_rolling_sum_7d', 
    #'mean_price_rolling_mean_28d',
       #'mean_price_rolling_sum_28d'   
]
evals_result={}

# Diccionario para guardar resultados por bucket
resultados_por_bucket = {}

# Grilla de hiperparámetros
param_grid = {
    'n_estimators': [200,350,500],
    'max_depth': [4, 6],
    'learning_rate': [0.05, 0.1],
    #'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
    #'reg_alpha': [0, 5],
    #'reg_lambda': [1, 5]
    "objective":["reg:squaredlogerror","reg:squarederror"]
}

for bucket in bins:
    data_bucket = ventas[ventas[separador] == bucket]
    
    # Crear test: últimas 7 filas por grupo
    test = data_bucket.groupby(['store_cod', 'subgroup_cod']).tail(7)
    train = data_bucket.drop(test.index)
    data_bucket.loc[data_bucket.index,"ps"]=-1
    data_bucket.loc[test.index,"ps"]=0

    test_fold=data_bucket["ps"]
    # Creamos el split
    ps = PredefinedSplit(test_fold)

    #definimos el modelo
    
    reg = XGBRegressor(
    #reducir el numero de columnas aumenta la importancia de std_price
    )

    grid_search = GridSearchCV(
        estimator=reg,
        param_grid=param_grid,
        scoring='neg_root_mean_squared_error',
        cv=ps,
        verbose=2,
        n_jobs=-1
    )
    
    grid_search.fit(data_bucket[FEATURES], data_bucket[TARGET])

    # Guardar resultados
    resultados_por_bucket[bucket] = {
        'best_params': grid_search.best_params_,
        'best_score': -grid_search.best_score_  # lo paso a positivo
    }
    
    print("Mejores parámetros:", grid_search.best_params_)
    print("Mejor RMSE en validación:", -grid_search.best_score_)    

C:\Users\emili\AppData\Local\Temp\ipykernel_11564\1824466024.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


Fitting 1 folds for each of 48 candidates, totalling 48 fits
Mejores parámetros: {'colsample_bytree': 1, 'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 500, 'objective': 'reg:squarederror'}
Mejor RMSE en validación: 2.311519698926529


C:\Users\emili\AppData\Local\Temp\ipykernel_11564\1824466024.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


Fitting 1 folds for each of 48 candidates, totalling 48 fits
Mejores parámetros: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 200, 'objective': 'reg:squarederror'}
Mejor RMSE en validación: 2.0809230770998046


C:\Users\emili\AppData\Local\Temp\ipykernel_11564\1824466024.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


...

Fitting 1 folds for each of 48 candidates, totalling 48 fits
Mejores parámetros: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 500, 'objective': 'reg:squarederror'}
Mejor RMSE en validación: 2.4271779823607127


C:\Users\emili\AppData\Local\Temp\ipykernel_11564\1824466024.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


Fitting 1 folds for each of 48 candidates, totalling 48 fits
Mejores parámetros: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 500, 'objective': 'reg:squaredlogerror'}
Mejor RMSE en validación: 2.5717356758920578


C:\Users\emili\AppData\Local\Temp\ipykernel_11564\1824466024.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


Fitting 1 folds for each of 48 candidates, totalling 48 fits
Mejores parámetros: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 200, 'objective': 'reg:squaredlogerror'}
Mejor RMSE en validación: 2.1824601949967866


In [65]:
resultados_por_bucket
acum=0
for key in resultados_por_bucket.keys():
    
    acum+=resultados_por_bucket[key]['best_score']
    print(key+ " rmse: "+str(resultados_por_bucket[key]['best_score']))
print(acum/len(resultados_por_bucket.keys()))

Men rmse: 2.311519698926529
Fiction rmse: 2.0809230770998046
Audio rmse: 2.0183582906692012
Computing rmse: 2.4648140231563977
Games rmse: 2.1968518354016546
Outdoor rmse: 2.411669001644258
Non-Fiction rmse: 2.0724137803376883
Team Sports rmse: 2.272388833569273
Educational rmse: 2.270911223251704
Skincare rmse: 2.0774338692251093
Cookware rmse: 2.0230864178977397
Appliances rmse: 2.0635550250064805
Children rmse: 2.1366228068923583
Fitness rmse: 2.1775094235977126
Furniture rmse: 1.9235999872193807
Pantry rmse: 2.1665689931245584
Beverages rmse: 2.1385269090448094
Mobile rmse: 1.8185567610476894
Snacks rmse: 2.5060655733276005
Action rmse: 2.1976055304499145
Haircare rmse: 2.4271779823607127
Makeup rmse: 2.5717356758920578
Women rmse: 2.1824601949967866
2.196102387571279


In [ ]:
from inline_sql import sql, sql_val

taka=sql^"""SELECT DISTINCT category, "group" FROM ventas order by category"""
taka["dict"]=taka["group"].map(resultados_por_bucket)
taka["rmse"]=0
taka["params"]={}
for i in range(len(taka)):
    fila=taka["dict"].iloc[i]
    taka["rmse"].iloc[i]=fila["best_score"]
    taka["params"].iloc[i]=fila["best_params"]
taka.to_csv("prueba_parametro_por_grupo.csv")

In [87]:
taka.to_csv("prueba_parametro_por_grupo.csv")

In [88]:
#randomized_search
#model=XGBRegressor()
separador="category"
bins=ventas[separador].unique()
TARGET=["demand"]
FEATURES=[#'mean_price', #'std_price', 
          'max_price', 'min_price',
        'cluster', #'store_cod', 
        # 'subgroup_cod', #'day',
       #'month', 
        'year', 'anual_sin_1',
    'anual_cos_1', 'mensual_sin_1',
       #'mensual_cos_1', 
        #'anual_sin_2', 'anual_cos_2', 
    'mensual_sin_2', #'demand_rolling_mean_7d',
       'demand_rolling_sum_7d', 
    #'demand_rolling_mean_28d',
       #'demand_rolling_sum_28d'
       #'mensual_cos_2',
        # 'mean_price_rolling_mean_7d',
       #'mean_price_rolling_sum_7d', 
    #'mean_price_rolling_mean_28d',
       #'mean_price_rolling_sum_28d'   
]
evals_result={}

# Diccionario para guardar resultados por bucket
resultados_por_bucket_rnd = {}

# Grilla de hiperparámetros
param_grid = {
    'n_estimators': [200,350,500],
    'max_depth': [4, 6],
    'learning_rate': [0.05, 0.1],
    #'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
    'reg_alpha': [0, 5,10,20],
    'reg_lambda': [1, 5],
    "objective":["reg:squaredlogerror","reg:squarederror"]
}

for bucket in bins:
    data_bucket = ventas[ventas[separador] == bucket]
    
    # Crear test: últimas 7 filas por grupo
    test = data_bucket.groupby(['store_cod', 'subgroup_cod']).tail(7)
    train = data_bucket.drop(test.index)
    data_bucket.loc[data_bucket.index,"ps"]=-1
    data_bucket.loc[test.index,"ps"]=0

    X_test=test[FEATURES]
    y_test=test[TARGET]

    test_fold=data_bucket["ps"]
    # Creamos el split
    ps = PredefinedSplit(test_fold)

    fit_params = {
    "eval_set": [(X_test, y_test)],
    "verbose": False
    }

    #definimos el modelo
    n_iter_search=10
    reg = XGBRegressor(
        early_stopping_rounds= 30,
        eval_metric= "rmse",
    #reducir el numero de columnas aumenta la importancia de std_price
    )


    rand_search = RandomizedSearchCV(
    estimator=reg,
    param_distributions=param_grid,
    n_iter=n_iter_search,
    scoring='neg_root_mean_squared_error',
    cv=ps,  # tu PredefinedSplit definido antes
    verbose=2,
    n_jobs=-1,
    random_state=42
)
    rand_search.fit(data_bucket[FEATURES], data_bucket[TARGET],**fit_params)

    # Guardar resultados
    resultados_por_bucket_rnd[bucket] = {
        'best_params': rand_search.best_params_,
        'best_score': -rand_search.best_score_  # lo paso a positivo
    }
    
    print("Mejores parámetros:", rand_search.best_params_)
    print("Mejor RMSE en validación:", -rand_search.best_score_)    



C:\Users\emili\AppData\Local\Temp\ipykernel_11564\974415837.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


Fitting 1 folds for each of 10 candidates, totalling 10 fits
Mejores parámetros: {'reg_lambda': 1, 'reg_alpha': 10, 'objective': 'reg:squarederror', 'n_estimators': 350, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 1}
Mejor RMSE en validación: 2.210301238790134


C:\Users\emili\AppData\Local\Temp\ipykernel_11564\974415837.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


Fitting 1 folds for each of 10 candidates, totalling 10 fits
Mejores parámetros: {'reg_lambda': 1, 'reg_alpha': 10, 'objective': 'reg:squarederror', 'n_estimators': 350, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 1}
Mejor RMSE en validación: 2.052879777195339


C:\Users\emili\AppData\Local\Temp\ipykernel_11564\974415837.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


...

Fitting 1 folds for each of 10 candidates, totalling 10 fits
Mejores parámetros: {'reg_lambda': 1, 'reg_alpha': 10, 'objective': 'reg:squarederror', 'n_estimators': 500, 'max_depth': 4, 'learning_rate': 0.1, 'colsample_bytree': 1}
Mejor RMSE en validación: 2.403744668373586


C:\Users\emili\AppData\Local\Temp\ipykernel_11564\974415837.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


Fitting 1 folds for each of 10 candidates, totalling 10 fits
Mejores parámetros: {'reg_lambda': 1, 'reg_alpha': 10, 'objective': 'reg:squarederror', 'n_estimators': 350, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 1}
Mejor RMSE en validación: 2.0006608073493375


C:\Users\emili\AppData\Local\Temp\ipykernel_11564\974415837.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bucket.loc[data_bucket.index,"ps"]=-1


Fitting 1 folds for each of 10 candidates, totalling 10 fits
Mejores parámetros: {'reg_lambda': 1, 'reg_alpha': 0, 'objective': 'reg:squarederror', 'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 0.8}
Mejor RMSE en validación: 2.288032235643285


In [ ]:
#clothing Mejores parámetros: {'objective': 'reg:squarederror', 'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.1, 'colsample_bytree': 1}

In [57]:
parameters=[
    {'reg_lambda': 1, 'reg_alpha': 10, 'objective': 'reg:squarederror', 'n_estimators': 500, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 1},
{'reg_lambda': 1, 'reg_alpha': 10, 'objective': 'reg:squarederror', 'n_estimators': 500, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 1},
 {'reg_lambda': 10, 'reg_alpha': 0, 'objective': 'reg:squarederror', 'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.1, 'colsample_bytree': 1},
 {'reg_lambda': 10, 'reg_alpha': 0, 'objective': 'reg:squarederror', 'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.1, 'colsample_bytree': 1},
{'reg_lambda': 1, 'reg_alpha': 5, 'objective': 'reg:squarederror', 'n_estimators': 350, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 0.8},
 {'reg_lambda': 1, 'reg_alpha': 5, 'objective': 'reg:squarederror', 'n_estimators': 350, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 0.8},
{'reg_lambda': 1, 'reg_alpha': 10, 'objective': 'reg:squarederror', 'n_estimators': 500, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 1},
{'reg_lambda': 1, 'reg_alpha': 5, 'objective': 'reg:squarederror', 'n_estimators': 350, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 0.8}]

print((2.21+2.05+2.11+2.23+2.31+2.40+2.00+2.28)/8)

2.19875


In [ ]:
from inline_sql import sql, sql_val

taka=sql^"""SELECT DISTINCT category FROM ventas order by category"""
taka["dict"]=taka["category"].map(resultados_por_bucket_rnd)
taka["rmse"]=0
taka["params"]={}
for i in range(len(taka)):
     fila=taka["dict"].iloc[i]
     taka["rmse"].iloc[i]=fila["best_score"]
     taka["params"].iloc[i]=fila["best_params"]
taka.to_csv("prueba_parametro_por_categoria.csv")
#resultados_por_bucket
taka

In [96]:
taka[["category","rmse"]]
#Beauty pesimo
#books ta weno
#clothing same
#Electronics	
#Groceries	
#Home & Kitchen	
#Sports	
#Toys

,category,rmse
0,Beauty,2.403745
1,Books,2.052880
2,Clothing,2.210301
3,Electronics,2.122012
4,Groceries,2.288032
5,Home & Kitchen,2.000661
6,Sports,2.313770
7,Toys,2.230675
